
## Without Zeeman and with orbital effects, spin-orbit has little effect on $I_c$

## With spin-orbit and with orbital effects, Zeeman has little effect on $I_c$

## Without orbital effects and without spin-orbit, the $B$ dependence scales both with $L$ and $\mu$

## In single mode regime, Zeeman has the predominant effect on $I_c$

## Both in the single mode and multimode regime, the orbital effect has the predominant effect on $I_c$

## $0-\pi$ transitions?
* $L=640, \mu=50$
* $L=640, \mu=40$
* $L=160, \mu=30$


In [ ]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
%matplotlib inline
# %config InlineBackend.figure_format = 'svg'

golden_mean = (np.sqrt(5) - 1) / 2 # Aesthetic ratio
fig_width_pt = 246.0 # Columnwidth
inches_per_pt = 1.0 / 72.27 # Convert pt to inches
fig_width = fig_width_pt * inches_per_pt
fig_height = fig_width * golden_mean # height in inches
fig_size = [fig_width, fig_height]

params = {'backend': 'ps',
          'axes.labelsize': 10,
          'font.size': 10,
          'legend.fontsize': 8,
          'xtick.labelsize': 8,
          'ytick.labelsize': 8,
          'text.usetex': True,
          'figure.figsize': fig_size,
          'font.family': 'serif',
          'font.serif': 'Computer Modern Roman',
          'legend.frameon': True,
          'savefig.dpi': 300
         }

plt.rcParams.update(params)
plt.rc('text.latex', preamble=[r'\usepackage{color}', r'\usepackage{bm}'])

import matplotlib.colors as colors
    
class HistogramNormalize(colors.Normalize):
    def __init__(self, data, vmin=None, vmax=None):
        if vmin is not None:
            data = data[data > vmin]
        if vmax is not None:
            data = data[data < vmax]
            
        self.sorted_data = np.sort(data.flatten())
        colors.Normalize.__init__(self, vmin, vmax)

    def __call__(self, value, clip=None):
        return np.ma.masked_array(np.searchsorted(self.sorted_data, value) /
                                  len(self.sorted_data))

# Multi mode regime:

In [ ]:
df = pd.read_hdf('data/all_data_circular_A_in_SM_and_SC.hdf')
gb = df.groupby(['T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu'])

total = gb.get_group((1, 640, True, True, 50, 20, 50))
Zeeman_only = gb.get_group((1, 640, True, False, 50, 20, 50))
orbital_only = gb.get_group((1, 640, True, True, 0, 20, 50))
no_spin_orbit = gb.get_group((1, 640, True, False, 50, 0, 50))

fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.125, 0.2, (0.95-0.125), (0.95-0.2)])
ax.plot(total['B_x'], total['current_c'], label='all effects')
ax.plot(Zeeman_only['B_x'], Zeeman_only['current_c'], label='$A=0$')
ax.plot(orbital_only['B_x'], orbital_only['current_c'], label='$g=0$')
ax.plot(no_spin_orbit['B_x'], no_spin_orbit['current_c'], label=r'$\alpha=0, \; A=0$')
ax.set_yticks(np.arange(0, 100, 10))
ax.set_ylim(0, 50)
ax.set_xlabel('Magnetic field $B$ [$T$]')
ax.set_ylabel('Critical current $I_c$ [meV$/\Phi_0$]')
ax.legend(framealpha=0.2)
# fig.savefig('figures/decay_length_2d.pdf', format='pdf', dpi=1000)
plt.show()

# Single mode regime:

In [ ]:
df = pd.read_hdf('data/all_data_circular_A_in_SM_and_SC.hdf')
gb = df.groupby(['T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu'])

total = gb.get_group((1, 640, True, True, 50, 20, 10))
Zeeman_only = gb.get_group((1, 640, True, False, 50, 20, 10))
orbital_only = gb.get_group((1, 640, True, True, 0, 20, 10))
no_spin_orbit = gb.get_group((1, 640, True, False, 50, 0, 10))

fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.125,0.2,(0.95-0.125),(0.95-0.2)])
ax.plot(total['B_x'], total['current_c'], label='all effects')
ax.plot(Zeeman_only['B_x'], Zeeman_only['current_c'], label='$A=0$')
ax.plot(orbital_only['B_x'], orbital_only['current_c'], label='$g=0$')
ax.plot(no_spin_orbit['B_x'], no_spin_orbit['current_c'], label=r'$\alpha=0, \; A=0$')
ax.set_yticks([0.0, 0.1, 0.2, 0.3])
ax.set_ylim(0, 0.35)
ax.set_xlabel('Magnetic field $B$ [$T$]')
ax.set_ylabel('Critical current $I_c$ [meV$/\Phi_0$]')
ax.legend(framealpha=0.2)
# fig.savefig('figures/decay_length_2d.pdf', format='pdf', dpi=1000)
plt.show()

# Both

In [ ]:
from scipy.signal import argrelextrema
df = pd.read_hdf('data/all_data_circular_A_in_SM_and_SC.hdf')
gb = df.groupby(['T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu'])
fig, axs = plt.subplots(2, 1, sharey=False, sharex=True, figsize=(fig_width, 2 * fig_height))
plt.subplots_adjust(bottom=0.2, left=0.125, right=0.80, top=0.9, hspace=0.10, wspace=0.15)

current = lambda orbital=True, g=50, alpha=20, mu=10: gb.get_group((1, 640, True, orbital, g, alpha, mu))['current_c'].as_matrix()
B_x = total['B_x']

def plot_curves(ax, mu):
    ax.plot(B_x, current(mu=mu), label='all effects', c='b')
    ax.plot(B_x, current(orbital=False, mu=mu), label=r'$\bm{A}=0$', c='g')
    ax.plot(B_x, current(g=0, mu=mu), label='$g=0$', c='r')
    ax.plot(B_x, current(alpha=0, mu=mu), label=r'$\alpha=0$', c='m')
    ax.plot(B_x, current(orbital=False, alpha=0, mu=mu), label=r'$\alpha=0, \; \bm{A}=0$', c='c')
    ax.plot(B_x, current(alpha=0, g=0, mu=mu), label=r'$\alpha=0, \; g=0$', c='y')
    return ax

axs[0] = plot_curves(axs[0], 10)
axs[0].set_yticks([0.0, 0.2, 0.4])
axs[0].set_ylim(0, 0.65)
axs[0].set_xlabel('')
axs[0].set_ylabel('$I_c$ [meV$/\Phi_0$]')
leg0 = axs[0].legend(framealpha=1)
axs[0].text(0.015, 0.9*0.65, '(a)')

axs[1] = plot_curves(axs[1], 50)
axs[1].set_yticks([0, 10, 20, 30])
axs[1].set_ylim(0, 42)
axs[1].set_xlabel('Magnetic field $B_x$ [$T$]')
axs[1].set_ylabel('$I_c$ [meV$/\Phi_0$]')
axs[1].text(0.015, 0.9*42, '(b)')

for i in argrelextrema(current(orbital=False, alpha=0, mu=10), np.less):
    x = B_x.iloc[i]
    y1, y2 = 0, 60
    axs[0].plot((x, x), (y1, y2), 'c--')

for i in argrelextrema(current(orbital=False, alpha=0, mu=50), np.less):
    x = B_x.iloc[i]
    y1, y2 = 0, 60
    axs[1].plot((x, x), (y1, y2), 'c--')

    
def first_min_max(arr):
    return B_x.iloc[argrelextrema(arr, np.less)[0][0]], B_x.iloc[argrelextrema(arr, np.greater)[0][0]]

B1, B2 = first_min_max(current(orbital=False, alpha=0, mu=10)) # for next cell
B3, B4 = first_min_max(current(orbital=False, alpha=0, mu=50)) # for next cell
fig.savefig('figures/critical_currents.pdf', format='pdf', dpi=600)
plt.show()

In [ ]:
from scipy.signal import argrelextrema
df = pd.read_hdf('data/all_data_circular_A_in_SM_and_SC.hdf')
gb = df.groupby(['B_x', 'T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu'])


def currents(B_x=0, orbital=True, g=50, alpha=20, mu=10):
    return gb.get_group((B_x, 1, 640, True, orbital, g, alpha, mu))['currents'].as_matrix()[0]

phases = gb.get_group((0, 1, 640, True, True, 50, 20, 10))['phases'].as_matrix()[0]

fig, axs = plt.subplots(2, 2, sharey=False, sharex=True, figsize=(fig_width, 2 * fig_height))
plt.subplots_adjust(bottom=0.2, left=0.125, right=0.80, top=0.9, hspace=0.10, wspace=0.15)

axs[0][0].plot(phases, currents(B_x=B1, mu=10), label='all effects', c='b')
axs[0][0].plot(phases, currents(B_x=B1, orbital=False, mu=10), label=r'$\bm{A}=0$', c='g')
axs[0][0].plot(phases, currents(B_x=B1, g=0, mu=10), label='$g=0$', c='r')
axs[0][0].plot(phases, currents(B_x=B1, alpha=0, mu=10), label=r'$\alpha=0$', c='m')
axs[0][0].plot(phases, currents(B_x=B1, orbital=False, alpha=0, mu=10), label=r'$\alpha=0, \; \bm{A}=0$', c='c')
axs[0][0].plot(phases, currents(B_x=B1, alpha=0, g=0, mu=10), label=r'$\alpha=0, \; g=0$', c='y')
axs[0][0].set_yticks([-0.4, 0.0, 0.4])
axs[0][0].set_ylim(-0.4, 0.4)
axs[0][0].set_xlim(-3.14, 3.14)
axs[0][0].set_xlabel('')
axs[0][0].set_ylabel('$I_c$ [meV$/\Phi_0$]')
axs[0][0].text(-3.00, 0.8*0.4, '(a)')

axs[1][0].plot(phases, currents(B_x=B2, mu=50), label='all effects', c='b')
axs[1][0].plot(phases, currents(B_x=B2, orbital=False, mu=50), label=r'$\bm{A}=0$', c='g')
axs[1][0].plot(phases, currents(B_x=B2, g=0, mu=50), label='$g=0$', c='r')
axs[1][0].plot(phases, currents(B_x=B2, alpha=0, mu=50), label=r'$\alpha=0$', c='m')
axs[1][0].plot(phases, currents(B_x=B2, orbital=False, alpha=0, mu=50), label=r'$\alpha=0, \; \bm{A}=0$', c='c')
axs[1][0].plot(phases, currents(B_x=B2, alpha=0, g=0, mu=50), label=r'$\alpha=0, \; g=0$', c='y')
axs[1][0].set_yticks([-30, -15, 0, 15, 30])
axs[1][0].set_ylim(-42, 42)
axs[1][0].set_xlabel('$\phi$ [rad]')
axs[1][0].set_ylabel('$I_c$ [meV$/\Phi_0$]')
axs[1][0].text(-3, 0.8*42, '(b)')

axs[0][1].plot(phases, currents(B_x=B3, mu=10), label='all effects', c='b')
axs[0][1].plot(phases, currents(B_x=B3, orbital=False, mu=10), label=r'$\bm{A}=0$', c='g')
axs[0][1].plot(phases, currents(B_x=B3, g=0, mu=10), label='$g=0$', c='r')
axs[0][1].plot(phases, currents(B_x=B3, alpha=0, mu=10), label=r'$\alpha=0$', c='m')
axs[0][1].plot(phases, currents(B_x=B3, orbital=False, alpha=0, mu=10), label=r'$\alpha=0, \; \bm{A}=0$', c='c')
axs[0][1].plot(phases, currents(B_x=B3, alpha=0, g=0, mu=10), label=r'$\alpha=0, \; g=0$', c='y')
axs[0][1].set_yticks([])
axs[0][1].set_ylim(-0.4, 0.4)
axs[0][1].set_xlim(-3.14, 3.14)
axs[0][1].set_xlabel('')
axs[0][1].set_ylabel('')
axs[0][1].text(-3.00, 0.8*0.4, '(c)')

axs[1][1].plot(phases, currents(B_x=B4, mu=50), label='all effects', c='b')
axs[1][1].plot(phases, currents(B_x=B4, orbital=False, mu=50), label=r'$\bm{A}=0$', c='g')
axs[1][1].plot(phases, currents(B_x=B4, g=0, mu=50), label='$g=0$', c='r')
axs[1][1].plot(phases, currents(B_x=B4, alpha=0, mu=50), label=r'$\alpha=0$', c='m')
axs[1][1].plot(phases, currents(B_x=B4, orbital=False, alpha=0, mu=50), label=r'$\alpha=0, \; \bm{A}=0$', c='c')
axs[1][1].plot(phases, currents(B_x=B4, alpha=0, g=0, mu=50), label=r'$\alpha=0, \; g=0$', c='y')
axs[1][1].set_yticks([])
axs[1][1].set_ylim(-42, 42)
axs[1][1].set_xlabel('$\phi$ [rad]')
axs[1][1].set_ylabel('')
axs[1][1].text(-3, 0.8*42, '(d)')
fig.savefig('figures/current_phase_relations.pdf', format='pdf', dpi=600)
plt.show()

# Critical phase

In [ ]:
df = pd.read_hdf('data/all_data_circular_A_in_SM_and_SC.hdf')
gb = df.groupby(['T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu'])

fig, axs = plt.subplots(2, 1, sharey=False, sharex=True, figsize=(fig_width, 2 * fig_height))
plt.subplots_adjust(bottom=0.2, left=0.125, right=0.80, top=0.9, hspace=0.10, wspace=0.15)

total_single =         gb.get_group((1, 640, True, True, 50, 20, 50))
total_multi =          gb.get_group((1, 640, True, True, 50, 20, 10))
no_orbital_single =    gb.get_group((1, 640, True, False, 50, 20, 50))
no_orbital_multi =     gb.get_group((1, 640, True, False, 50, 20, 10))
no_zeeman_single =     gb.get_group((1, 640, True, True, 0, 20, 50))
no_zeeman_multi =      gb.get_group((1, 640, True, True, 0, 20, 10))
no_spin_orbit_no_orbital_single = gb.get_group((1, 640, True, False, 50, 0, 50))
no_spin_orbit_no_orbital_multi =  gb.get_group((1, 640, True, False, 50, 0, 10))

B_x = total['B_x']

axs[0].plot(B_x, total_single['phase_c'], label='all effects')
axs[0].plot(B_x, no_orbital_single['phase_c'], label=r'$\bm{A}=0$')
axs[0].plot(B_x, no_zeeman_single['phase_c'], label='$g=0$')
axs[0].plot(B_x, no_spin_orbit_no_orbital_single['phase_c'], label=r'$\alpha=0, \; \bm{A}=0$')
axs[0].set_yticks([-3, 0, 3])
axs[0].set_ylim(-3, 3)
axs[0].set_xlabel('')
axs[0].set_ylabel('$\phi_c$ [rad]')
axs[0].legend(framealpha=0.2)
axs[0].text(0.02, 0.8*3, '(a)')


axs[1].plot(B_x, total_multi['phase_c'], label='all effects')
axs[1].plot(B_x, no_orbital_multi['phase_c'], label=r'$\bm{A}=0$')
axs[1].plot(B_x, no_zeeman_multi['phase_c'], label='$g=0$')
axs[1].plot(B_x, no_spin_orbit_no_orbital_multi['phase_c'], label=r'$\alpha=0, \; \bm{A}=0$')
axs[1].set_yticks([-3, 0, 3])
axs[1].set_ylim(-3, 3)
axs[1].set_xlabel('Magnetic field $B_x$ [$T$]')
axs[1].set_ylabel('$\phi_c$ [rad]')
axs[1].legend(framealpha=0.2)
axs[1].text(0.02, 0.8*3, '(b)')
fig.savefig('figures/critical_currents.pdf', format='pdf', dpi=600)
plt.show()

# Gate dependence

In [ ]:
df = pd.read_hdf('data/all_gate_data_current_as_function_of_gate_and_B_x__A_in_SM_and_SC.hdf')
gb = df.groupby(['gate_size'])
gr = gb.get_group((80))

fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (gr.min()['B_x'], gr.max()['B_x'], gr.min()['Vs'], gr.max()['Vs'])
norm = HistogramNormalize(np.array(gr['current_c']))
kwargs = dict(extent=extent, cmap='viridis', aspect='auto', vmax=20, 
              norm=norm, origin='lower')
im = ax.imshow(gr.pivot(index='B_x', columns='Vs', values='current_c').T, **kwargs)
ax.set_xlabel('Magnetic field $B_x$ [$T$]')
ax.set_ylabel(r'$V_\textrm{bias}$ [meV]')
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cbar_ticks = [0, 5, 10, 15, 20, 25]
cb = fig.colorbar(im, cax=cax, extend='max', label=r'$I_c$ [meV$/\Phi_0$]', ticks=cbar_ticks)
cb.ax.set_yticklabels(cbar_ticks)
fig.savefig('figures/gate_dependence.pdf', format='pdf', dpi=600)

# Rotating field

In [ ]:
df = pd.read_hdf('data/rotation_of_field__L_640_nm__infinite_leads.hdf')
df['theta'] = (df['theta'] * 180 / np.pi).round(0).astype(int)
gb = df.groupby(['theta', 'T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu'])

theta_0 = gb.get_group((0, 1, 640, True, True, 50, 20, 50))
theta_30 = gb.get_group((30, 1, 640, True, True, 50, 20, 50))
theta_60 = gb.get_group((60, 1, 640, True, True, 50, 20, 50))
theta_90 = gb.get_group((90, 1, 640, True, True, 50, 20, 50))

B_x = theta_0['B']

fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.125, 0.2, (0.95-0.125), (0.95-0.2)])
ax.plot(B_x, theta_0['current_c'], label=r'$\bm{B} \parallel \hat{x}$')
ax.plot(B_x, theta_90['current_c'], label=r'$\bm{B} \parallel \hat{z}$')
ax.plot(B_x, theta_30['current_c'], label=r'$\bm{B} \parallel \sqrt{3} \hat{x} + \hat{z}$')
ax.plot(B_x, theta_60['current_c'], label=r'$\bm{B} \parallel \hat{x} + \sqrt{3} \hat{z}$')
ax.set_yticks(np.arange(0, 100, 10))
ax.set_ylim(0, 40)
ax.set_xlabel(r'Magnetic field $|\bm{B}|$ [$T$]')
ax.set_ylabel('$I_c$ [meV$/\Phi_0$]')
ax.legend(framealpha=1)
fig.savefig('figures/rotation_of_field.pdf', format='pdf', dpi=1000)
plt.show()

$U H(B, \phi) U^\dagger = H(B, -\phi), \quad U = \sigma_x \delta(x+x')$

In [ ]:
df = pd.read_hdf('data/no_disorder_B_vs_I_c__L_640_nm__infinite_leads__varying_mu.hdf')
gb = df.groupby(['orbital', 'g', 'alpha'])

gr = gb.get_group((True, 50, 20))
for (orbital, g, alpha), gr in gb:
    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
    extent = (gr.min()['B_x'], gr.max()['B_x'], gr.min()['mu'], gr.max()['mu'])
    norm = HistogramNormalize(np.array(gr['current_c']))
    kwargs = dict(extent=extent, cmap='viridis', aspect='auto', vmax=20, 
                  norm=norm, origin='lower')
    im = ax.imshow(gr.pivot_table(index='B_x', columns='mu', values='current_c').T, **kwargs)
    ax.set_xlabel('Magnetic field $B_x$ [$T$]')
    ax.set_ylabel(r'$\mu$ [meV]')
    ax.set_title(r'$\bm{{A}} {}0, \; g={}, \; \alpha={}$'.format((r'\ne' if orbital else '='), g, alpha))
    cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
    cbar_ticks = [0, 5, 10, 15, 20, 25]
    cb = fig.colorbar(im, cax=cax, extend='max', label=r'$I_c$ [meV$/\Phi_0$]', ticks=cbar_ticks)
    cb.ax.set_yticklabels(cbar_ticks)
    plt.show()